In [1]:
# Step 1: libraries import
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Lambda, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

In [2]:
text = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance."""

In [3]:
text

'Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.'

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
word2idx = tokenizer.word_index
vocab_size = len(word2idx) + 1


In [5]:
seq = tokenizer.texts_to_sequences([text])[0]

In [6]:
pairs = []
window = 2  # context words around target word
for i, target in enumerate(seq):
    for j in range(max(0, i - window), min(len(seq), i + window + 1)):
        if i != j:
            pairs.append((seq[j], target))

contexts = np.array([x[0] for x in pairs])
targets = np.array([x[1] for x in pairs])
targets = to_categorical(targets, vocab_size)  # target words one-hot encode

In [7]:
# Step 4: Create CBOW Model
input_Layer = Input(shape=(1,))
# Embedding(input_dim, output_dim, input_length)
embedding_Layer = Embedding(vocab_size, 8, input_length=1, name="embedding")(input_Layer)
x = Lambda(lambda x: K.mean(x, axis=1))(embedding_Layer)
output_Layer = Dense(vocab_size, activation='softmax')(x)

model = Model(inputs=input_Layer, outputs=output_Layer)
model.compile(optimizer='adam', loss='categorical_crossentropy')

C:\Users\rahul\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
# Step 5: Model training
model.fit(contexts, targets, epochs=10)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.3161
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.3104 
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.3053 
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.3001 
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.2948 
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.2897 
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.2839 
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.2782 
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 4.2719  
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.2655 


In [28]:
# Testing prediction on a context word
test_word = "artificial"
test_idx = np.array([[word2idx[test_word]]])
pred = model.predict(test_idx)
predicted_idx = np.argmax(pred)

for w, i in word2idx.items():
    if i == predicted_idx:
        predicted_word = w
        break

print(f"\n Context word: '{test_word}'")
print(f"Predicted Target word: '{predicted_word}'")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

 Context word: 'artificial'
Predicted Target word: 'networks'
